# Stream Filtering - KinesisAnalytics V2 API

Experiment: can we use Kinesis data analytics to filter the records written to one stream to populate other streams? Or more accurately, how do we do this, and what's the latency for a record written to the main stream to hit the filtered stream?

## Setup

First, we need some streams

In [ ]:
import boto3

kinesis_client = boto3.client('kinesis')

In [ ]:
import os
account_no = os.environ['ACCOUNT_NO']

In [ ]:
# Create some streams
main_stream_response = kinesis_client.create_stream(
    StreamName='main', 
    ShardCount = 1)

In [ ]:
kinesis_client.describe_stream(StreamName='main')

In [ ]:
kinesis_client.create_stream(StreamName='filtered', ShardCount=1)

In [ ]:
kinesis_client.describe_stream(StreamName='filtered')

In [ ]:
from datetime import datetime, timezone

def timestamp():
    the_time = datetime.now(timezone.utc)
    return the_time.isoformat()

## Stream Write

In [ ]:
import uuid

event = {
    "specversion":"1.0",
    "type":"newFoo",
    "source":"foo",
    "id":str(uuid.uuid4()),
    "time":timestamp(),
    "data":{"fooaddr":"foostuffval",
           "foolist": [1,2,3],
           "barobj": {
               "baraatr1":"yes",
               "barattr2":False,
               "barattr3":122.22
           }}
}

In [ ]:
print(event)

In [ ]:
import json

prr = kinesis_client.put_record(
    StreamName='main',
    Data=json.dumps(event).encode(),
    PartitionKey=event['source']
)

In [ ]:
prr

## Analytics App

### Role

In [ ]:
kinesis_app_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "ReadInputKinesis",
            "Effect": "Allow",
            "Action": [
                "kinesis:DescribeStream",
                "kinesis:GetShardIterator",
                "kinesis:GetRecords"
            ],
            "Resource": [
                "arn:aws:kinesis:us-east-1:" + account_no + ":stream/main"
            ]
        },
        {
            "Sid": "WriteOutputKinesis",
            "Effect": "Allow",
            "Action": [
                "kinesis:DescribeStream",
                "kinesis:PutRecord",
                "kinesis:PutRecords"
            ],
            "Resource": [
                "arn:aws:kinesis:us-east-1:" + account_no + ":stream/filtered"
            ]
        }
    ]
}

In [ ]:
kinesis_app_policy

In [ ]:
assume_role_policy = {
    "Statement":[{
        "Effect":"Allow",
        "Principal": {"Service":["kinesisanalytics.amazonaws.com"]},
        "Action": ["sts:AssumeRole"]
    }]
}
    
    
import json

json.dumps(assume_role_policy)

In [ ]:
iam = boto3.client('iam')

crr = iam.create_role(
    RoleName='sample-ka-app-role',
    Path='/service-role/',
    AssumeRolePolicyDocument=json.dumps(assume_role_policy)
)

print(crr)

In [ ]:
prp = iam.put_role_policy(
    RoleName='sample-ka-app-role',
    PolicyName='KAPolicy',
    PolicyDocument=json.dumps(kinesis_app_policy)
)

print(prp)

### App Definition

In [ ]:
ka = boto3.client('kinesisanalyticsv2')

In [ ]:
application_code = '-- ** Continuous Filter ** \n-- Performs a continuous filter based on a WHERE condition.\n--          .----------.   .----------.   .----------.              \n--          |  SOURCE  |   |  INSERT  |   |  DESTIN. |              \n-- Source-->|  STREAM  |-->| & SELECT |-->|  STREAM  |-->Destination\n--          |          |   |  (PUMP)  |   |          |              \n--          \'----------\'   \'----------\'   \'----------\'               \n-- STREAM (in-application): a continuously updated entity that you can SELECT from and INSERT into like a TABLE\n-- PUMP: an entity used to continuously \'SELECT ... FROM\' a source STREAM, and INSERT SQL results into an output STREAM\n-- Create output stream, which can be used to send to a destination\nCREATE OR REPLACE STREAM "DESTINATION_SQL_STREAM" ("payload" VARCHAR(32000));\n-- Create pump to insert into output \nCREATE OR REPLACE PUMP "STREAM_PUMP" AS INSERT INTO "DESTINATION_SQL_STREAM"\n-- Select all columns from source stream\nSELECT STREAM "payload"\nFROM "SOURCE_SQL_STREAM_001"\n-- LIKE compares a string to a string pattern (_ matches all char, % matches substring)\n-- SIMILAR TO compares string to a regex, may use ESCAPE\nWHERE "type" = \'newFoo\';\n'
print(application_code)

In [ ]:
application_config = {
    "SqlApplicationConfiguration" : {
        "Inputs":[{
            'NamePrefix': 'SOURCE_SQL_STREAM',
            "KinesisStreamsInput": {
                "ResourceARN":"arn:aws:kinesis:us-east-1:" + account_no + ":stream/main"
            },
            "InputParallelism": {
                "Count": 1
            },
            "InputSchema": {
                "RecordFormat": {
                    "RecordFormatType":"JSON",
                    'MappingParameters': {
                        'JSONMappingParameters': {
                            'RecordRowPath': '$'
                        }
                    }
                },
                'RecordEncoding': 'UTF-8',
                'RecordColumns': [
                    {
                        'Name': 'type',
                        'Mapping': '$.type',
                         'SqlType': 'VARCHAR(64)'
                    },
                    {
                        'Name': 'payload', 
                        'Mapping': '$', 
                        'SqlType': 'VARCHAR(32000)'
                    }]
            }
        }],
        "Outputs": [
            {
                "Name":"DESTINATION_SQL_STREAM",
                "KinesisStreamsOutput": {
                    "ResourceARN":"arn:aws:kinesis:us-east-1:" + account_no + ":stream/filtered"
                },
                "DestinationSchema": {
                    'RecordFormatType': 'JSON'
                }
            }
        ]
    },
    "ApplicationCodeConfiguration": {
        'CodeContentType': 'PLAINTEXT',
        "CodeContent": {    
            'TextContent': application_code
        }
    }
}

print(application_config)

In [ ]:
car = ka.create_application(
    ApplicationName='dave',
    ApplicationDescription='dave app',
    RuntimeEnvironment='SQL-1_0',
    ServiceExecutionRole="arn:aws:iam::" + account_no + ":role/service-role/sample-ka-app-role",
    ApplicationConfiguration = application_config
)

In [ ]:
print(car)
print(car['ApplicationDetail']['CreateTimestamp'])
create_timestamp = car['ApplicationDetail']['CreateTimestamp']

In [ ]:
sa = ka.start_application(
    ApplicationName='dave',
    RunConfiguration={
        'SqlRunConfigurations':[
            {
                'InputId':'1.1',
                'InputStartingPositionConfiguration': {
                    'InputStartingPosition':'NOW'
                }
            }
        ]
    }
)
print(sa)

## Stream Read

In [ ]:
## Read from stream

shardId = prr['ShardId']
print('shard id is %s' % shardId)

gsir = kinesis_client.get_shard_iterator(
    StreamName='main',
    ShardId=shardId,
    ShardIteratorType='TRIM_HORIZON'
)
print(gsir)

In [ ]:
## Read from currne position of the iterator
grr = kinesis_client.get_records(
    ShardIterator=gsir['ShardIterator']
)

print(grr)

In [ ]:
records = grr['Records']
for r in records:
    print(r)

## Cleanup

In [ ]:
ka.stop_application(
    ApplicationName='dave'
)

In [ ]:
ka.delete_application(
    ApplicationName='dave',
    CreateTimestamp=create_timestamp
)

In [ ]:
kinesis_client.delete_stream(StreamName='main')
kinesis_client.delete_stream(StreamName='filtered')

In [ ]:
kinesis_client.list_streams()

In [ ]:
iam.delete_role_policy(
    RoleName='sample-ka-app-role',
    PolicyName='KAPolicy'
)

In [ ]:
iam.delete_role(
    RoleName='sample-ka-app-role'
)